In [1]:
import numpy as np
from qkeras import QDense, QActivation
from qkeras.quantizers import binary
from tensorflow.keras.models import Sequential

model = Sequential()

model.add(QDense(5, input_shape=(5,),
                 kernel_quantizer=binary(),
                 name='dense1'))

model.add(QActivation('binary', name='act1'))

model.add(QDense(5,
                 kernel_quantizer=binary(),
                 name='dense2'))

model.add(QActivation('binary', name='act2'))

model.summary()


2024-11-14 18:57:51.567889: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-14 18:57:51.589531: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-14 18:57:51.589553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-14 18:57:51.590362: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 18:57:51.594382: I tensorflow/core/platform/cpu_feature_guar

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense1 (QDense)             (None, 5)                 30        
                                                                 
 act1 (QActivation)          (None, 5)                 0         
                                                                 
 dense2 (QDense)             (None, 5)                 30        
                                                                 
 act2 (QActivation)          (None, 5)                 0         
                                                                 
Total params: 60 (240.00 Byte)
Trainable params: 60 (240.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [2]:
class Quantizer:
    """
    Base class for representing quantizers in hls4ml.

    Subclasses of ``Quantizer`` are expected to wrap the quantizers of upstream tools (e.g., QKeras).

    Args:
        bits (int): Total number of bits used by the quantizer.
        hls_type (NamedType): The hls4ml type used by the quantizer.
    """

    def __init__(self, bits, hls_type):
        self.bits = bits
        self.hls_type = hls_type

    def __call__(self, data):
        raise NotImplementedError


class BinaryQuantizer(Quantizer):
    """Quantizer that quantizes to 0 and 1 (``bits=1``) or -1 and 1 (``bits==2``).

    Args:
        bits (int, optional): Number of bits used by the quantizer. Defaults to 2.

    Raises:
        Exception: Raised if ``bits>2``
    """

    def __init__(self, bits=2):

        super().__init__(bits, hls_type=None)

    def __call__(self, data):
        zeros = np.zeros_like(data)
        ones = np.ones_like(data)
        quant_data = data
        if self.bits == 1:
            quant_data = np.where(data > 0, ones, zeros).astype('int')
        if self.bits == 2:
            quant_data = np.where(data > 0, ones, -ones)
        return quant_data

w2_data = model.get_layer('dense1').get_weights()
w2_quantizer = BinaryQuantizer(bits=1)
w2_quantized = w2_quantizer(w2_data[0])
print(w2_quantized)
print(np.array([0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1]).reshape(5,5))


[[0 1 1 0 1]
 [0 1 1 1 1]
 [1 0 0 1 0]
 [0 1 1 0 0]
 [0 1 1 0 0]]
[[0 1 1 0 0]
 [1 0 1 1 1]
 [0 0 1 1 0]
 [0 0 0 1 1]
 [0 1 0 0 1]]


In [4]:
from hls4ml.converters import convert_from_keras_model
from hls4ml.utils.config import config_from_keras_model

# 設定輸入形狀
input_shape = (None, 5)  # MNIST 數據集的形狀

# 從模型生成配置
config = config_from_keras_model(model, backend='Vitis')

# 設定輸出目錄
output_dir = 'hls4ml_bnn_prj_keras'

# 轉換為 HLS 模型
hls_model_init = convert_from_keras_model(
    model,
    output_dir=output_dir,
    hls_config=config,
    backend='Vitis'
)

Interpreting Sequential
Topology:
Layer name: dense1_input, layer type: InputLayer, input shapes: [[None, 5]], output shape: [None, 5]
Layer name: dense1, layer type: QDense, input shapes: [[None, 5]], output shape: [None, 5]
Layer name: act1, layer type: Activation, input shapes: [[None, 5]], output shape: [None, 5]
Layer name: dense2, layer type: QDense, input shapes: [[None, 5]], output shape: [None, 5]
Layer name: act2, layer type: Activation, input shapes: [[None, 5]], output shape: [None, 5]


In [5]:
hls_model_init.compile()

Writing HLS project
hls4ml_bnn_prj_keras
hls4ml_config.yml
Done


/home/robin/miniconda3/envs/keras/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
